# Sentiment Evaluation on Testset

Compare human annotated labels to rule-based assignment (using the policy: summing polarity scores from SentiWS) on Posts/Comments in Dev-/Testset.

In [1]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## Loading

In [ ]:
data_path = "/mnt/DATA/NRW2019 Dropbox/data 4good/CSVData/TestData"
testset_fpath = os.path.join(data_path, ".csv") #FIXME

testset = pd.read_csv(testset_fpath)

## Evaluation

# Conclusion

#FIXME